<a href="https://colab.research.google.com/github/tsparaskevas/ML_EDDE2/blob/main/Final%20essay/scrape_newssites_for_final_EDDE2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Τι να πρέπει να κάνω εγώ**
* Να επιλέγω ποιο website θέλω να scrapάρω
* Να επιλέγω πόσων ημερών άρθρα θέλω να scrapάρω
* Να επιλέγω αν θέλω και ολόκληρα τα άρθρα εκτός από τα teasers

**Τι κάνει το πρόγραμα**
* Διαλέγει τον κατάλληλο scrapper ανάλογα με το website
* Τρέχει τον scrapper
* Σώζει το αποτέλεσμα σαν csv στο drive

**Πώς δουλεύει το πρόγραμμα**
* Κοιτάει αν υπάρχει στη βάση δεδομένων (websites-info) το ζητούμενο website
  * Αν δεν υπάρχει τυπώνει: "Δεν υπάρχουν πληροφορίες για το <όνομα site>" και σταματάει
  * Αν υπάρχει προχωράει παρακάτω
* Κοιτάει αν υπάρχει Φάκελος στο Drive με το όνομα του site
  * Αν δεν υπάρχει τον φτιάχνει και συνεχίζει
  * Αν υπάρχει συνεχίζει
* Φτιάχνει τις αντίστοιχες μεταβλητές
* Κοιτάει στο website-info:
  * πώς προχωράνε οι σελίδες (pagination) στη ροή των ειδήσεων, στο συγκεκριμένο site
  * πόσες σελίδες πιάνουν οι ειδήσεις μιας μέρας
* Τρέχει τον κατάλληλο scrapper
* Φτιάχνει το dataframe με τα δεδομένα των σκραπαρισμένων άρθρων και:
  * Αν υπήρχε Φάκελος στο Drive φέρνει σαν dataframe το csv με suffix all και κάνει σε αυτό append το dataframe που έφτιαξε σήμερα
  * Αν δεν υπήρχε Φάκελος στο Drive σώζει το σημερινό dataframe στο Drive KAI σαν csv με suffix all

**Τι είναι και τι περιέχει το websites-info**
* Είναι df
* Περιέχει:
  *  

###**ENTER CUSTOM VALUES**

**Μεταβλητές που ΠΡΕΠΕΙ να συμπληρώνοται πάντα στην αρχή**

In [ ]:
# 1. WEBSITE (str)
website_name = 'newsit'
website_suf = "gr"

# 2. ΠΟΣΩΝ ΗΜΕΡΩΝ ΑΡΘΡΑ ΘΕΛΩ ΝΑ ΦΕΡΩ (int)
days_to_scrape = 0 # If set to other than 0, leave start_page and last_page to 0

# 3. ΑΠΟ ΠΟΙΑ ΣΕΛΙΔΑ ΤΩΝ ΤΕASERS ΝΑ ΑΡΧΙΣΕΙ ΤΟ SCRAPING? (int)
start_page = 52 # If days_to_scrape is set to other than 0, leave this to 1

# 4. ΜΕΧΡΙ ΠΟΙΑ ΣΕΛΙΔΑ ΤΩΝ TEASERS ΝΑ ΓΙΝΕΙ SCRAPING? (int)
last_page = 53 # If days_to_scrape is set to other than 0, leave this to 0

# 5. ΘΕΛΩ FULL ARTICLES? (str[yes/no])
full_wanted = 'yes'

# ΠΟΣΕΣ ΣΕΛΙΔΕΣ ΠΙΑΝΕΙ ΜΙΑ ΗΜΕΡΑ 
pages_per_day = 5

**Folders**

In [ ]:
# main drive path
main_drive_path = "gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2"

# topic folder
topic_folder = 'homocide'

####**Define Search pages and Keywords**

Define search term (may be a combination of 2 or more words e.g. "εργατικό ατύχημα")

In [ ]:
# ανθρωποκτονία OR δολοφονία OR συζυγοκτονία OR ανθρωποκτόνος OR δολοφόνος OR συζυγοκτόνος

In [ ]:
search_term = "ανθρωποκτονία"

AUTO FIX SEARCH TERM

In [ ]:
if ' ' in search_term:
  termsL = search_term.split()
  search_term = '+'.join(termsL)
else:
  search_term = search_term
print(search_term)

ανθρωποκτονία


**(needed for naftemboriki.gr)**

Define start and last date

In [ ]:
start_date = "2020-01-01"
end_date = "2020-12-31"

**Check inputs**

In [ ]:
# days_to scrape, start_page, last_page
import sys
if days_to_scrape == 0:
  if start_page == 0 or last_page == 0 or start_page > last_page:
    sys.exit("Something's WRONG with days_to_scrape / start_page / last_page")
  else:
    print("Values of variables 2, 3 & 4 seem RIGHT")
else:
  print("Values of variables 2, 3 & 4 seem RIGHT")

Values of variables 2, 3 & 4 seem RIGHT


###**Import Libraries**

In [ ]:
from datetime import date 
from datetime import datetime
import sys

import os
from pathlib import Path
import pandas as pd

import re

import requests
from bs4 import BeautifulSoup
import time

from google.colab import drive 
drive.mount('gdrive') 

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [ ]:
#efsyn_ppd = 14
#news247_ppd = 5

###**Prepare**

**Auto calculate Variables**

In [ ]:
# ΗΜΕΡΟΜΗΝΙΑ scraping
today = str(date.today())

# CSV NAMES and PATHS
main_drive_path = main_drive_path
website_folder = website_name + "." + website_suf 
website_path = main_drive_path + "/" + topic_folder

csv_teasers_name = website_name + "_teasers_" + today + ".csv"
csv_teasers_path_and_name = website_path + '/' + csv_teasers_name

csv_full_articles_name = website_name + "_full_articles_" + today + ".csv"
csv_full_articles_path_and_name = website_path + '/' + csv_full_articles_name

csv_teasers_all_name = website_name + "_teasers_all.csv"
csv_teasers_all_path_and_name = website_path + '/' + csv_teasers_all_name

csv_full_articles_all_name = website_name + "_full_articles_all.csv"
csv_full_articles_all_path_and_name = website_path + '/' + csv_full_articles_all_name

# START AND LAST PAGES TO BE SCRAPED
if days_to_scrape > 0:
  start_page = 1
  last_page = (days_to_scrape * 5) + 1
elif start_page == 0 or last_page == 0 or start_page > last_page:
  sys.exit("Something's wrong with days_to_scrape / start_page / last_page")
else:
  last_page += 1
print(f'Start page is {start_page} and Last page is {last_page - 1}')

# FUNCTIONS' NAMES
scrape_search_pages_func = "scrape_" + website_name + "_search_pages" # for scraping Search Results teasers pages
scrape_pages_func = "scrape_" + website_name + "_latest_pages" # for scraping teasers pages
scrape_articles_func = "scrape_" + website_name + "_full_articles" # for scraping full articles pages

Start page is 52 and Last page is 53


**Create website folder if NOT exists**

In [ ]:
# IF website's folder NOT EXISTS -> Create website's folder
if not os.path.exists(website_path):
  os.makedirs(website_path)
  print(f"'{website_path}' folder is created")
else:
  print(f"'{website_path}' folder already exists")

'gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2/homocide' folder already exists


**Import csvs**

In [ ]:
# Import websites' scraping info csv


# Import website's all full articles csv
if os.path.exists(csv_full_articles_all_path_and_name):
  full_articles_all_df = pd.read_csv(csv_full_articles_all_path_and_name)
  print(f'"{csv_full_articles_all_name}" is imported as "full_articles_all_df"')
else:
  print(f'"{csv_full_articles_all_name}" doesn\'t exist yet')

# Import website's all teasers csv
if os.path.exists(csv_teasers_all_path_and_name):
  teasers_all_df = pd.read_csv(csv_teasers_all_path_and_name)
  print(f'"{csv_teasers_all_name}" is imported as "teasers_all_df"')
else:
  print(f'"{csv_teasers_all_name}" doesn\'t exist yet')

"efsyn_full_articles_all.csv" doesn't exist yet
"efsyn_teasers_all.csv" doesn't exist yet


**Load scrappers.ipynb**

In [ ]:
full_articles_list = []

In [ ]:
#drive.mount("gdrive", force_remount=True)

Mounted at gdrive


In [ ]:
%run '/content/gdrive/MyDrive/Colab Notebooks/Scraping/greek_news_sites_scrappers_search.ipynb' 

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


###**Execute**

**Teasers**

In [ ]:
# Scrape latest pages
globals()[scrape_search_pages_func](start_page, last_page)

Now Scraping page: 52
Now Scraping page: 53
# of dictionaries created: 40


In [ ]:
# Create dataframe
teasers_df = pd.DataFrame(teaser_articles_list)
teasers_df

,Url,Title,Date
0,https://www.newsit.gr/egklhma/lasithi-ithele-n...,"Ήθελε να αυτοκτονήσει μετά το άγριο φονικό - ""...",2020-02-13 13:40:17
1,https://www.newsit.gr/egklhma/lasithi-paradoth...,Στον ανακριτή ο δράστης του άγριου φονικού στο...,2020-02-13 09:57:16
2,https://www.newsit.gr/topikes-eidhseis/patra-s...,"Στο ""κόκκινο"" η αγωνία για τον φανοποιό - Παρα...",2020-02-12 15:50:12
3,https://www.newsit.gr/topikes-eidhseis/thessal...,Θεσσαλονίκη: Καταδικάστηκε ερήμην ο Τσέχος οδη...,2020-02-11 19:23:22
4,https://www.newsit.gr/topikes-eidhseis/patra-i...,Πάτρα: Η πιο μεγάλη ώρα για τον φανοποιό! Η α...,2020-02-11 10:03:00
5,https://www.newsit.gr/topikes-eidhseis/korinth...,Κόρινθος: Νέες αποκαλύψεις για τους ληστές τρα...,2020-02-09 16:21:32
6,https://www.newsit.gr/topikes-eidhseis/serres-...,Σέρρες: Η “πόρτα” στο καφενείο τον έκανε δολοφ...,2020-02-07 11:16:23
7,https://www.newsit.gr/egklhma/el-as-1-397-syll...,1.397 συλλήψεις σε δύο μόλις εβδομάδες από την...,2020-02-06 19:03:51
8,https://www.newsit.gr/kosmos/malaysia-airlines...,Πτήση ΜΗ17 - Εξελίξεις για την συντριβή του μο...,2020-02-03 18:48:20
9,https://www.newsit.gr/egklhma/dionysos-ston-ei...,Με μάσκα οξυγόνου πήγε στον εισαγγελέα ο 77χρο...,2020-01-27 19:56:53


In [ ]:
# Save csv
teasers_df.to_csv(csv_teasers_all_path_and_name, index=False)

In [ ]:
teasers_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2/suicide/newsit_teasers_all.csv")

In [ ]:
# Conditional saving csv
#if not os.path.exists(csv_teasers_all_path_and_name): # if website is being scrapped for the first time
#  teasers_df.to_csv(csv_teasers_all_path_and_name, index=False)
#else:                                                 # if website's teaser pages have been scrapped again in the past
#  teasers_all_df = teasers_all_df.append(teasers_df, ignore_index=True)
#  teasers_all_df.drop_duplicates(subset=['Url'], inplace=True, ignore_index=True)
#  teasers_all_df.sort_values('Datetime', ascending=False, inplace=True, ignore_index=True)
#  teasers_all_df.to_csv(csv_teasers_all_path_and_name, index=False)

**Full articles** (if full_wanted = 'yes')

In [ ]:
full_articles_list = []

In [ ]:
# Scrape full articles
if full_wanted == 'yes':
  globals()[scrape_articles_func]()

Scraping article 1 of 40
Scraping article 2 of 40
Scraping article 3 of 40
Scraping article 4 of 40
Scraping article 5 of 40
Scraping article 6 of 40
Scraping article 7 of 40
Scraping article 8 of 40
Scraping article 9 of 40
Scraping article 10 of 40
Scraping article 11 of 40
Scraping article 12 of 40
Scraping article 13 of 40
Scraping article 14 of 40
Scraping article 15 of 40
Scraping article 16 of 40
Scraping article 17 of 40
Scraping article 18 of 40
Scraping article 19 of 40
Scraping article 20 of 40
Scraping article 21 of 40
Scraping article 22 of 40
Scraping article 23 of 40
Scraping article 24 of 40
Scraping article 25 of 40
Scraping article 26 of 40
Scraping article 27 of 40
Scraping article 28 of 40
Scraping article 29 of 40
Scraping article 30 of 40
Scraping article 31 of 40
Scraping article 32 of 40
Scraping article 33 of 40
Scraping article 34 of 40
Scraping article 35 of 40
Scraping article 36 of 40
Scraping article 37 of 40
Scraping article 38 of 40
Scraping article 39 o

In [ ]:
full_articles_df = pd.DataFrame(full_articles_list)

In [ ]:
# Save csv
full_articles_df.to_csv(csv_full_articles_all_path_and_name, index=False)

In [ ]:
# Μεγιστοποίηση του πλάτους της στήλης για εμφάνιση ολόκληρου του κειμένου
#pd.set_option('display.max_colwidth', None)
# Reset display options#
#pd.reset_option("all")

In [ ]:
full_articles_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2/homocide/newsit_full_articles_dolofonia_all.csv')

In [ ]:
full_articles_df#.loc[1040:1060]

,Title,Url,Lead,Section,Date,Author,Text
0,"Ήθελε να αυτοκτονήσει μετά το άγριο φονικό - ""...",https://www.newsit.gr/egklhma/lasithi-ithele-n...,,Εγκλημα,2020-02-13 13:40:17,,Να βάλει τέλος στη ζωή του ήθελε ο δράστης του...
1,Στον ανακριτή ο δράστης του άγριου φονικού στο...,https://www.newsit.gr/egklhma/lasithi-paradoth...,,Εγκλημα,2020-02-13 09:57:16,,Τον 43χρονο συνόδευε ο δικηγόρος του με τον οπ...
2,"Στο ""κόκκινο"" η αγωνία για τον φανοποιό - Παρα...",https://www.newsit.gr/topikes-eidhseis/patra-s...,,Τοπικα Νεα,2020-02-12 15:50:12,,Πέντε μέρες μετά την διαφωνία εισαγγελέα και α...
3,Θεσσαλονίκη: Καταδικάστηκε ερήμην ο Τσέχος οδη...,https://www.newsit.gr/topikes-eidhseis/thessal...,,Τοπικα Νεα,2020-02-11 19:23:22,,Φυλάκιση τεσσάρων ετών και δύο μηνών στον οδηγ...
4,Πάτρα: Η πιο μεγάλη ώρα για τον φανοποιό! Η α...,https://www.newsit.gr/topikes-eidhseis/patra-i...,,Τοπικα Νεα,2020-02-11 10:03:00,,Συνεδριάζει σήμερα το Συμβούλιο Πλημμελειοδικώ...
5,Κόρινθος: Νέες αποκαλύψεις για τους ληστές τρα...,https://www.newsit.gr/topikes-eidhseis/korinth...,,Τοπικα Νεα,2020-02-09 16:21:32,,Στην εξάρθρωση μιας επικίνδυνης εγκληματικής ο...
6,Σέρρες: Η “πόρτα” στο καφενείο τον έκανε δολοφ...,https://www.newsit.gr/topikes-eidhseis/serres-...,,Τοπικα Νεα,2020-02-07 11:16:23,,Σε συνολική ποινή κάθειρξης 15 ετών και 9 μηνώ...
7,1.397 συλλήψεις σε δύο μόλις εβδομάδες από την...,https://www.newsit.gr/egklhma/el-as-1-397-syll...,,Εγκλημα,2020-02-06 19:03:51,,Σε 1.397 συλλήψεις προχώρησαν οι Υπηρεσίες της...
8,Πτήση ΜΗ17 - Εξελίξεις για την συντριβή του μο...,https://www.newsit.gr/kosmos/malaysia-airlines...,,Κοσμος,2020-02-03 18:48:20,,Οι εισαγγελικές αρχές της Ολλανδίας απήγγειλαν...
9,Με μάσκα οξυγόνου πήγε στον εισαγγελέα ο 77χρο...,https://www.newsit.gr/egklhma/dionysos-ston-ei...,,Εγκλημα,2020-01-27 19:56:53,,Προθεσμία για να απολογηθεί την Πέμπτη πήρε ο ...


In [ ]:
newsit_test_dolofonia_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2/homocide/newsit_full_articles_dolofonia_all.csv")

In [ ]:
#full_articles_all_df['Datetime'] = pd.to_datetime(full_articles_all_df['Datetime'])
#full_articles_all_df.info()

In [ ]:
# Create dataframe
#if full_wanted == 'yes':
#  full_articles_df = pd.DataFrame(full_articles_list)
  # Conditional saving csv
#  if not os.path.exists(csv_full_articles_all_path_and_name): # if website is being scrapped for the first time
#    full_articles_df.to_csv(csv_full_articles_all_path_and_name, index=False)
#  else:                                                 # if website's full articles have been scrapped again in the past
#    full_articles_all_df = full_articles_all_df.append(full_articles_df, ignore_index=True)
#    full_articles_all_df.drop_duplicates(subset=['Article url'], inplace=True, ignore_index=True)
#    full_articles_all_df.sort_values('Datetime', ascending=False, inplace=True, ignore_index=True)
#    full_articles_all_df.to_csv(csv_full_articles_all_path_and_name, index=False)

###**Concat newsit csvs**

In [ ]:
df1 = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2/homocide/newsit_full_articles_anthropoktonia1_all.csv")
df2 = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2/homocide/newsit_full_articles_anthropoktonia2_all.csv")
df = pd.concat([df1, df2], axis=0)
print(df.shape)
df.drop_duplicates(subset=['Title'], keep="last", inplace=True)
df.reset_index(drop=True, inplace=True)
df

(380, 7)


,Title,Url,Lead,Section,Date,Author,Text
0,Αιτωλοακαρνανία: Δεμένο με καλώδιο ήταν το ηλι...,https://www.newsit.gr/topikes-eidhseis/aitoloa...,,Τοπικα Νεα,2021-01-14 16:47:00,,NaN
1,Βρυξέλλες: Άγρια επεισόδια σε διαδήλωση για το...,https://www.newsit.gr/kosmos/vrykselles-agria-...,,Κοσμος,2021-01-13 23:59:35,,"Η βελγική αστυνομία προχώρησε την Τετάρτη, 13...."
2,Πήρε προθεσμία ο ψευτογιατρός: Απολογείται τέλ...,https://www.newsit.gr/egklhma/pire-prothesmia-...,,Εγκλημα,2021-01-11 13:50:12,,Στις 28 Ιανουαρίου θα απολογηθεί τελικά συμπλη...
3,Νέα προθεσμία για τον 47χρονο ψευτογιατρό – Η ...,https://www.newsit.gr/ellada/nea-prothesmia-gi...,,Ελλαδα,2021-01-11 12:32:34,,Σε συμπληρωματική απολογία κλήθηκε την Δευτέρα...
4,Στον ανακριτή σήμερα ο ψευτογιατρός – Οι κατηγ...,https://www.newsit.gr/ellada/ston-anakriti-sim...,,Ελλαδα,2021-01-11 06:34:39,,Σε συμπληρωματική απολογία έχει κληθεί σήμερα ...
...,...,...,...,...,...,...,...
335,Συγκλονίζει η τραγική φιγούρα της δολοφονίας τ...,https://www.newsit.gr/egklhma/dolofonia-neas-a...,,Εγκλημα,2019-12-25 16:57:49,,Να δει την μητέρα του και τον πατέρα ζητά ο μι...
336,"""Από τις κλωτσιές του δολοφόνου το παιδί γεννή...",https://www.newsit.gr/egklhma/patra-apo-tis-kl...,,Εγκλημα,2019-12-24 20:25:28,,Δεν σταματούν να σοκάρουν οι αποκαλύψεις για τ...
337,Ξεσπά κατά του φονιά ο πατέρας της 33χρονης Αδ...,https://www.newsit.gr/egklhma/patra-ksespa-kat...,,Εγκλημα,2019-12-24 16:59:27,,"Η Πάτρα κήδευσε την 33χρονη Αδαμαντία Αντύπα, ..."
338,Στη φυλακή ο 54χρονος συζυγοκτόνος της Νέας Αλ...,https://www.newsit.gr/egklhma/irakleio-sti-fyl...,,Εγκλημα,2019-12-24 15:01:30,,Την ώρα που στην Πάτρα γινόταν η κηδεία της Αδ...


In [ ]:
df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2/homocide/newsit_full_articles_anthropoktonia_all.csv", index=False)

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2/homocide/newsit_full_articles_anthropoktonia_all.csv")
df

,Title,Url,Lead,Section,Date,Author,Text
0,Αιτωλοακαρνανία: Δεμένο με καλώδιο ήταν το ηλι...,https://www.newsit.gr/topikes-eidhseis/aitoloa...,,Τοπικα Νεα,2021-01-14 16:47:00,,NaN
1,Βρυξέλλες: Άγρια επεισόδια σε διαδήλωση για το...,https://www.newsit.gr/kosmos/vrykselles-agria-...,,Κοσμος,2021-01-13 23:59:35,,"Η βελγική αστυνομία προχώρησε την Τετάρτη, 13...."
2,Πήρε προθεσμία ο ψευτογιατρός: Απολογείται τέλ...,https://www.newsit.gr/egklhma/pire-prothesmia-...,,Εγκλημα,2021-01-11 13:50:12,,Στις 28 Ιανουαρίου θα απολογηθεί τελικά συμπλη...
3,Νέα προθεσμία για τον 47χρονο ψευτογιατρό – Η ...,https://www.newsit.gr/ellada/nea-prothesmia-gi...,,Ελλαδα,2021-01-11 12:32:34,,Σε συμπληρωματική απολογία κλήθηκε την Δευτέρα...
4,Στον ανακριτή σήμερα ο ψευτογιατρός – Οι κατηγ...,https://www.newsit.gr/ellada/ston-anakriti-sim...,,Ελλαδα,2021-01-11 06:34:39,,Σε συμπληρωματική απολογία έχει κληθεί σήμερα ...
...,...,...,...,...,...,...,...
335,Συγκλονίζει η τραγική φιγούρα της δολοφονίας τ...,https://www.newsit.gr/egklhma/dolofonia-neas-a...,,Εγκλημα,2019-12-25 16:57:49,,Να δει την μητέρα του και τον πατέρα ζητά ο μι...
336,"""Από τις κλωτσιές του δολοφόνου το παιδί γεννή...",https://www.newsit.gr/egklhma/patra-apo-tis-kl...,,Εγκλημα,2019-12-24 20:25:28,,Δεν σταματούν να σοκάρουν οι αποκαλύψεις για τ...
337,Ξεσπά κατά του φονιά ο πατέρας της 33χρονης Αδ...,https://www.newsit.gr/egklhma/patra-ksespa-kat...,,Εγκλημα,2019-12-24 16:59:27,,"Η Πάτρα κήδευσε την 33χρονη Αδαμαντία Αντύπα, ..."
338,Στη φυλακή ο 54χρονος συζυγοκτόνος της Νέας Αλ...,https://www.newsit.gr/egklhma/irakleio-sti-fyl...,,Εγκλημα,2019-12-24 15:01:30,,Την ώρα που στην Πάτρα γινόταν η κηδεία της Αδ...


###**Check completion**

**Search full articles**

**Teasers**

In [ ]:
csv_teasers_all_path_and_name

'gdrive/MyDrive/Colab Notebooks/Scraping/CSVs/ErgasiaEDDE2/troxaio/newsbomb_teasers_all.csv'

In [ ]:
teasers_all_df = pd.read_csv(csv_teasers_all_path_and_name) # Import teasers_all as dataframe
print(teasers_all_df.shape)
teasers_all_df.info()

(39, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Url      39 non-null     object
 1   Title    39 non-null     object
 2   Lead     39 non-null     object
 3   Section  39 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


**Full articles**

In [ ]:
if full_wanted == 'yes': # If full articles were scrapped in the current session
  full_articles_all_df = pd.read_csv(csv_full_articles_all_path_and_name) # Import full_articles_all as dataframe
print(full_articles_all_df.shape)
full_articles_all_df.info()

(39, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    39 non-null     object
 1   Url      39 non-null     object
 2   Lead     39 non-null     object
 3   Section  39 non-null     object
 4   Date     39 non-null     object
 5   Author   39 non-null     object
 6   Text     39 non-null     object
dtypes: object(7)
memory usage: 2.3+ KB
